In [ ]:
from Solutions import Solution
import numpy as np
import random
import time
from pymoo.core.mutation import Mutation
from pymoo.core.variable import get, Real
from pymoo.operators.crossover.binx import mut_binomial
from pymoo.operators.repair.to_bound import set_to_bounds_if_outside
np.random.seed(0)

def mut_pm(X, xl, xu, eta, prob, at_least_once):
    # Just input each of the parameters as integers or floats. They'll convert to np.arrays
    # later, look up Pymoo GitHub and simplify this so that it just accepts single values
    X = np.expand_dims(X,axis=1)
    xl = np.asarray([xl])
    xu = np.asarray([xu])
    eta = np.asarray([eta])
    prob = np.asarray([prob])


    n, n_var = X.shape
    assert len(eta) == n
    assert len(prob) == n

    Xp = np.full(X.shape, np.inf)

    mut = mut_binomial(n, n_var, prob, at_least_once=at_least_once)
    mut[:, xl == xu] = False

    Xp[:, :] = X

    _xl = np.repeat(xl[None, :], X.shape[0], axis=0)[mut]
    _xu = np.repeat(xu[None, :], X.shape[0], axis=0)[mut]

    X = X[mut]
    eta = np.tile(eta[:, None], (1, n_var))[mut]

    delta1 = (X - _xl) / (_xu - _xl)
    delta2 = (_xu - X) / (_xu - _xl)

    mut_pow = 1.0 / (eta + 1.0)

    rand = np.random.random(X.shape)
    mask = rand <= 0.5
    mask_not = np.logical_not(mask)

    deltaq = np.zeros(X.shape)

    xy = 1.0 - delta1
    val = 2.0 * rand + (1.0 - 2.0 * rand) * (np.power(xy, (eta + 1.0)))
    d = np.power(val, mut_pow) - 1.0
    deltaq[mask] = d[mask]

    xy = 1.0 - delta2
    val = 2.0 * (1.0 - rand) + 2.0 * (rand - 0.5) * (np.power(xy, (eta + 1.0)))
    d = 1.0 - (np.power(val, mut_pow))
    deltaq[mask_not] = d[mask_not]

    # mutated values
    _Y = X + deltaq * (_xu - _xl)

    # back in bounds if necessary (floating point issues)
    _Y[_Y < _xl] = _xl[_Y < _xl]
    _Y[_Y > _xu] = _xu[_Y > _xu]

    # set the values for output
    Xp[mut] = _Y

    # in case out of bounds repair (very unlikely)
    Xp = set_to_bounds_if_outside(Xp, xl, xu)

    return Xp

def mutation(soln: Solution, pm_pixels: float, zero_prob: float, m_eta: int):
  # the nonzero pixels
  z_indices = np.where(np.sum(abs(soln.values),axis=1)>0)[0]
  # how many is z?
  z_len = len(z_indices)
  # how many is d = pm_pixels * z?
  d_len = int(np.floor(pm_pixels * z_len)) # this many pixels will change

  # how many is z_len - d_len?
  # A_count = z_len - d_len # these pixels will not change in mutation

  # now, pick d pixels and place them in any indices that are not in the variable z_indices
  old_d_indices = np.random.choice(z_indices, size = (d_len))
  new_d_indices = np.random.choice(np.where(np.sum(abs(soln.values),axis=1)==0)[0], size = (d_len))

  ones_prob = (1 - zero_prob) / 2

  soln.values[old_d_indices] = 0
  soln.values[new_d_indices] = np.random.choice([-1, 1, 0], size=(d_len, 3), p=(ones_prob, ones_prob, zero_prob))

  ### p_size ###

  soln.p_size = mut_pm(X = soln.p_size, xl = 0.0, xu = 1.0, eta = m_eta, prob = 1.0, at_least_once = False)[0]


def sbx(parent1geno, parent2geno, pc, nc, length):
    """
    SBX crossover operator
    :param parent1geno:
    :param parent2geno:
    :param pc:
    :param nc:
    :param length:
    :return:
    """ # pc is 1.0, nc is eta, length is max(dimensions)
    rs = [random.random() for _ in range(length)]
    uis = [random.random() for _ in range(length)]
    child1geno = parent1geno[:]
    child2geno = parent1geno[:]

    for g1, g2, c in zip(parent1geno, parent2geno, range(length)):
        if rs[c] < pc:
            bqi = (2 * uis[c]) ** (1 / (nc + 1)) if uis[c] < 0.5 else (1 / (2 * (1 - uis[c]))) ** (1 / (nc + 1))
            child1geno[c] = 0.5 * ((1 + bqi) * g1 + (1 - bqi) * g2)
            child2geno[c] = 0.5 * ((1 - bqi) * g1 + (1 + bqi) * g2)

    child1geno[child1geno<0]=0
    child2geno[child2geno>1]=1

    return child1geno, child2geno


def crossover(soln1: Solution, soln2: Solution, pc_pixels: float, c_eta: int):
    ### Crossover pixel locations ###
    l = max([int(len(soln1.pixels) * pc_pixels), 1]) # Think about it, right now pc_pixels (10%) is multiplying by a constant factor 50,000. So I'd say, either keep it this way and increase pc_pixels, or l needs to be a percentage of nonzero pixels not all and then crossover with those nonzero pixels
    # k = len(soln1.pixels)
    # S1 crossover with S2
    # 1. Generate set of different pixels in S2

    # Let's try this out:
    # delta = np.asarray([pi for pi in range(k) if soln2.pixels[pi] not in soln1.pixels])

    # offspring1 = soln1.copy()
    # if len(delta)>0:
    #     l = l if l <= len(delta) else len(delta)
    #     switched_pixels = np.random.choice(delta, size=(l,))
    #     offspring1.pixels[switched_pixels] = soln2.pixels[switched_pixels].copy()
    #     offspring1.values[switched_pixels] = soln2.values[switched_pixels].copy()

    # # S2 crossover with S1
    # # 1. Generate set of different pixels in S2
    # delta = np.asarray([pi for pi in range(k) if soln1.pixels[pi] not in soln2.pixels])
    # offspring2 = soln1.copy()
    # if len(delta)>0:
    #     l = l if l <= len(delta) else len(delta)
    #     switched_pixels = np.random.choice(delta, size=(l,))
    #     offspring2.pixels[switched_pixels] = soln1.pixels[switched_pixels].copy()
    #     offspring2.values[switched_pixels] = soln1.values[switched_pixels].copy()

    # Let's try some swapping random pixels. Maybe increase pc_pixels? Perhaps comment this out actually
    offspring1 = soln1.copy()
    switched_pixels = np.random.choice(soln2.pixels, size=(l,))
    offspring1.values[switched_pixels] = soln2.values[switched_pixels].copy()

    offspring2 = soln2.copy()
    switched_pixels = np.random.choice(soln1.pixels, size=(l,))
    offspring2.values[switched_pixels] = soln1.values[switched_pixels].copy()

    ### Crossover pixel intensities ###
    offspring1.p_size, offspring2.p_size = sbx(soln1.p_size.copy(), soln2.p_size.copy(), pc=1.0, nc=c_eta, length = 1)

    return offspring1, offspring2


def generate_offspring(parents, pc_pixels,pm_pixels, zero_prob, m_eta, c_eta):
    children = []
    # start_offspring = time.time()
    for pi in parents:
        # start_crossover = time.time()
        offspring1, offspring2 = crossover(pi[0], pi[1], pc_pixels = pc_pixels, c_eta = c_eta)
        # print("crossover:",time.time()-start_crossover)
        # start_mutation1 = time.time()
        mutation(offspring1,pm_pixels, zero_prob, m_eta = m_eta)
        # print("mutation1:",time.time()-start_mutation1)
        # start_mutation2 = time.time()
        mutation(offspring2,pm_pixels, zero_prob, m_eta = m_eta)
        # print("mutation2:",time.time()-start_mutation2)
        print(" ")

        assert len(np.unique(offspring1.pixels)) == len(offspring1.pixels)
        assert len(np.unique(offspring2.pixels)) == len(offspring2.pixels)
        children.extend([offspring1, offspring2])
    # print("offspring:",time.time()-start_offspring)
    return children


# def generate_offspring_(parents, pc_pixels,pm_pixels, all_pixels, zero_prob):
#     children = []
#     for pi in parents:
#         offspring1, offspring2 = pi[0].copy(), pi[1].copy()
#         mutation(offspring1,pm_pixels, all_pixels, zero_prob)
#         mutation(offspring2,pm_pixels, all_pixels, zero_prob)

#         assert len(np.unique(offspring1.pixels)) == len(offspring1.pixels)
#         assert len(np.unique(offspring2.pixels)) == len(offspring2.pixels)
#         children.extend([offspring1, offspring2])

#     return children



